In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Differential directions for mESC_Ag data

In [ ]:
%load_ext tensorboard

In [ ]:
from IPython import display
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
#from tensorflow import keras
import tensorflow as tf
import tensorflow_probability as tfp
import time
import pickle
import os

#Load Data
Root_Folder = "/content/drive/MyDrive"
data = "gastr"
data_parent_folder = "/Data"
code_parent_folder = "/scMVI"

os.chdir('/content/drive/My Drive/' + code_parent_folder +'/Code')

Met_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Met_train_" + data + ".csv",",")
CpG_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/CpG_train_" + data + ".csv",",")
Acc_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Acc_train_" + data + ".csv",",")
GpC_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/GpC_train_" + data + ".csv",",")
Rna_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_train_" + data + ".csv",",")
Rna_nrm_train=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_nrm_train_" + data + ".csv",",")

Met_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Met_test_" + data + ".csv",",")
CpG_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/CpG_test_" + data + ".csv",",")
Acc_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Acc_test_" + data + ".csv",",")
GpC_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/GpC_test_" + data + ".csv",",")
Rna_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_test_" + data + ".csv",",")
Rna_nrm_test=pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/Rna_nrm_test_" + data + ".csv",",")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
Rna = pd.concat([Rna_train,Rna_test],axis=0)

In [ ]:
MMet_train=Met_train.drop(columns=Met_train.columns[0])
MCpG_train=CpG_train.drop(columns=CpG_train.columns[0])
MAcc_train=Acc_train.drop(columns=Acc_train.columns[0])
MGpC_train=GpC_train.drop(columns=GpC_train.columns[0])
MRna_train=Rna_train.drop(columns=Rna_train.columns[0])
MRna_nrm_train=Rna_nrm_train.drop(columns=Rna_nrm_train.columns[0])

MMet_test=Met_test.drop(columns=Met_test.columns[0])
MCpG_test=CpG_test.drop(columns=CpG_test.columns[0])
MAcc_test=Acc_test.drop(columns=Acc_test.columns[0])
MGpC_test=GpC_test.drop(columns=GpC_test.columns[0])
MRna_test=Rna_test.drop(columns=Rna_test.columns[0])
MRna_nrm_test=Rna_nrm_test.drop(columns=Rna_nrm_test.columns[0])

In [ ]:
Met_tr=tf.convert_to_tensor(MMet_train,dtype=tf.float32)
Met_tst=tf.convert_to_tensor(MMet_test,dtype=tf.float32)
CpG_tr=tf.convert_to_tensor(MCpG_train,dtype=tf.float32)
CpG_tst=tf.convert_to_tensor(MCpG_test,dtype=tf.float32)
Acc_tr=tf.convert_to_tensor(MAcc_train,dtype=tf.float32)
Acc_tst=tf.convert_to_tensor(MAcc_test,dtype=tf.float32)
GpC_tr=tf.convert_to_tensor(MGpC_train,dtype=tf.float32)
GpC_tst=tf.convert_to_tensor(MGpC_test,dtype=tf.float32)
Rna_tr=tf.convert_to_tensor(MRna_train,dtype=tf.float32)
Rna_tst=tf.convert_to_tensor(MRna_test,dtype=tf.float32)
Rna_nrm_tr = tf.repeat(tf.math.log(tf.convert_to_tensor(MRna_nrm_train,dtype=tf.float32)),repeats=Met_tr.shape[1],axis=1)
Rna_nrm_tst= tf.repeat(tf.math.log(tf.convert_to_tensor(MRna_nrm_test,dtype=tf.float32)),repeats=Met_tr.shape[1],axis=1)

training_size=Rna_tr.shape[0]
test_size = Rna_tst.shape[0]
batch_size =  10

inp_met_sz = Met_tr.shape[1]
inp_acc_sz = Acc_tr.shape[1]
inp_rna_sz = Rna_tr.shape[1]

print(training_size)
print(test_size)
print(inp_met_sz,inp_acc_sz,inp_rna_sz)

636
190
6330 6330 6330


In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((Met_tr,CpG_tr,Acc_tr,GpC_tr,Rna_tr,Rna_nrm_tr)).shuffle(training_size).batch(batch_size))
val = tf.data.Dataset.from_tensor_slices((Met_tst,CpG_tst,Acc_tst,GpC_tst,Rna_tst,Rna_nrm_tst))
val_data = tf.data.Dataset.zip((val,)).batch(test_size)

In [ ]:
from Models import scMVI_NMT
from Obj_Blocks import *

In [ ]:
#Define model and load weights for the trained model
mdl = scMVI_NMT(inp_met_dim = inp_met_sz,inp_acc_dim=inp_acc_sz,inp_rna_dim=inp_rna_sz,intermidiate_dim=100,latent_dimensions=10)
mdl.load_weights(Root_Folder + code_parent_folder + "/Networks/scMVI_scNMT_"+data+"/scMVI_scNMT_"+data)

In [ ]:
#Functions for estimating differential directions and stats for experiments outlined in the main chapter 
def index_creation(meta_data,group_by,group_1,smps_group_1,group_2,smps_group_2):


                        cell_num = meta_data.shape[0]
                        group_index_1 = np.zeros((cell_num,))
                        group_index_2 = np.zeros((cell_num,))

                        ind_1 = np.where(meta_data[group_by]==group_1)[0]
                        ind_2 = np.where(meta_data[group_by]==group_2)[0]

                        if np.logical_or(len(ind_1)<smps_group_1,len(ind_2)<smps_group_2):

                                    raise ValueError('samples for groups 1 and 2 cannot be more than cells in groups 1 and 2')        



                        perm_ind_1 = ind_1[np.random.permutation(len(ind_1))[:smps_group_1]]
                        perm_ind_2 = ind_2[np.random.permutation(len(ind_2))[:smps_group_2]]

                        group_index_1[perm_ind_1]=1
                        group_index_2[perm_ind_2]=1

                        group_index_1 = tf.squeeze(tf.where(group_index_1==1))
                        group_index_2 = tf.squeeze(tf.where(group_index_2==1))

                        return group_index_1, group_index_2

def create_exp(metadata,grp_1,smps_1,grp_2,smps_2):

        smps_group_1 =[smps_1,int(np.round(smps_1*0.2)),smps_1,int(np.round(smps_1*0.2)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.5)),int(np.round(smps_1*0.2))]
        smps_group_2 =[smps_2,smps_2,int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2)),int(np.round(smps_2*0.5)),int(np.round(smps_2*0.2))]

        col_all_met = []
        col_all_acc = []
        col_all_rna = []

        for jj in range(5):


              group_index_11,group_index_21 = index_creation(metadata,'stage',grp_1,smps_group_1[0],grp_2,smps_group_2[0])
              group_index_12,group_index_22 = index_creation(metadata,'stage',grp_1,smps_group_1[1],grp_2,smps_group_2[1])
              group_index_13,group_index_23 = index_creation(metadata,'stage',grp_1,smps_group_1[2],grp_2,smps_group_2[2])
              group_index_14,group_index_24 = index_creation(metadata,'stage',grp_1,smps_group_1[3],grp_2,smps_group_2[3])
              group_index_15,group_index_25 = index_creation(metadata,'stage',grp_1,smps_group_1[4],grp_2,smps_group_2[4])
              group_index_16,group_index_26 = index_creation(metadata,'stage',grp_1,smps_group_1[5],grp_2,smps_group_2[5])
              group_index_17,group_index_27 = index_creation(metadata,'stage',grp_1,smps_group_1[6],grp_2,smps_group_2[6])

              a1_met,a1_acc,a1_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_11,group_index_21,'all')
              a2_met,a2_acc,a2_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_12,group_index_22,'all')
              a3_met,a3_acc,a3_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_13,group_index_23,'all')
              a4_met,a4_acc,a4_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_14,group_index_24,'all')
              a5_met,a5_acc,a5_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_15,group_index_25,'all')
              a6_met,a6_acc,a6_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_16,group_index_26,'all')
              a7_met,a7_acc,a7_rna = mdl.Diff_Dir_Dist_Avg(xxxx,group_index_17,group_index_27,'all')


              col_all_met.append([a1_met,a2_met,a3_met,a4_met,a5_met,a6_met,a7_met])
              col_all_acc.append([a1_acc,a2_acc,a3_acc,a4_acc,a5_acc,a6_acc,a7_acc])
              col_all_rna.append([a1_rna,a2_rna,a3_rna,a4_rna,a5_rna,a6_rna,a7_rna])

              print(100*(1+jj)/5.0)


        return col_all_met,col_all_acc,col_all_rna

def collect_stats(x):

        x_med = tfp.stats.percentile(tf.squeeze(x),50,axis=2)
        x_med_mn = tf.reduce_mean(tf.reduce_mean(x_med,axis=1),axis=0)[tf.newaxis,:]
        x_med_std = tf.math.sqrt(tf.reduce_mean(tfp.stats.variance(x_med,sample_axis=1),axis=0))[tf.newaxis,:]
        x_std = tf.math.sqrt(tf.reduce_mean(tfp.stats.variance(tf.squeeze(x),sample_axis=2),axis=0))

        x_med_rng = tf.reduce_mean(tf.reduce_max(x_med,axis=1) -tf.reduce_min(x_med,axis=1),axis=0)[tf.newaxis,:]

        return x_med_mn,x_med_std,x_std,x_med_rng


In [ ]:
Met_comb = tf.concat([Met_tr,Met_tst],axis=0)
CpG_comb = tf.concat([CpG_tr,CpG_tst],axis=0)
Acc_comb = tf.concat([Acc_tr,Acc_tst],axis=0)
GpC_comb = tf.concat([GpC_tr,GpC_tst],axis=0)
Rna_comb = tf.concat([Rna_tr,Rna_tst],axis=0)

Rna_nrm_comb = tf.concat([Rna_nrm_tr,Rna_nrm_tst],axis=0)

xxxx = [Met_comb,CpG_comb,Acc_comb,GpC_comb,Rna_comb,Rna_nrm_comb]

In [ ]:
#Load Metadata
Rna = pd.concat([Rna_train,Rna_test],axis=0)

gastr_ind_drp = [0,1,2,3,4,5,6,7,8,9]#8 
gastr_QC = pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/sample_metadata_gastr.csv",'\t',index_col=[0])


gastr_QC = gastr_QC.drop(columns=gastr_QC.columns[gastr_ind_drp],axis=1)
gastr_QC.index = gastr_QC.index.str.replace('-','.')

gastr_QC = gastr_QC.loc[Rna.iloc[:,0]]

gastr_ind_drp = [0,1,2,3,4,5,6,7,9,10]#8 
gastr_QC1 = pd.read_csv(Root_Folder + data_parent_folder + "/Data_Gastr/sample_metadata_gastr.csv",'\t',index_col=[0])


gastr_QC1 = gastr_QC1.drop(columns=gastr_QC1.columns[gastr_ind_drp],axis=1)
gastr_QC1.index = gastr_QC1.index.str.replace('-','.')

gastr_QC1 = gastr_QC1.loc[Rna.iloc[:,0]]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(gastr_QC1.value_counts())

stage
E7.5     343
E6.5     271
E5.5     108
E4.5     104
dtype: int64


In [ ]:
#Estimates differential directions
lab1 = "E7.5" #(343)
smps_1 = 343
lab2 = "E6.5" #(271)
smps_2 = 271
lab3 = "E5.5" #(108)
smps_3 = 108
lab4 = "E4.5" #(104)
smps_4 = 104

col_all_1_met,col_all_1_acc,col_all_1_rna = create_exp(gastr_QC1,lab1,smps_1,lab2,smps_2)
col_all_2_met,col_all_2_acc,col_all_2_rna = create_exp(gastr_QC1,lab2,smps_2,lab3,smps_3)
col_all_3_met,col_all_3_acc,col_all_3_rna = create_exp(gastr_QC1,lab3,smps_3,lab4,smps_4)

20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0
20.0
40.0
60.0
80.0
100.0


In [ ]:
cl_1_med_met,_,_,cl_1_med_rng_met = collect_stats(col_all_1_met)
cl_2_med_met,_,_,cl_2_med_rng_met = collect_stats(col_all_2_met)
cl_3_med_met,_,_,cl_3_med_rng_met = collect_stats(col_all_3_met)

cl_1_med_acc,_,_,cl_1_med_rng_acc = collect_stats(col_all_1_acc)
cl_2_med_acc,_,_,cl_2_med_rng_acc = collect_stats(col_all_2_acc)
cl_3_med_acc,_,_,cl_3_med_rng_acc = collect_stats(col_all_3_acc)

cl_1_med_rna,_,_,cl_1_med_rng_rna = collect_stats(col_all_1_rna)
cl_2_med_rna,_,_,cl_2_med_rng_rna = collect_stats(col_all_2_rna)
cl_3_med_rna,_,_,cl_3_med_rng_rna = collect_stats(col_all_3_rna)

In [ ]:
med_1_rna_rng = pd.DataFrame(np.squeeze(cl_1_med_rng_rna),index=Rna.columns[1:])
med_2_rna_rng = pd.DataFrame(np.squeeze(cl_2_med_rng_rna),index=Rna.columns[1:])
med_3_rna_rng= pd.DataFrame(np.squeeze(cl_2_med_rng_rna),index=Rna.columns[1:])

med_1_rna = pd.DataFrame(np.squeeze(cl_1_med_rna),index=Rna.columns[1:])
med_2_rna = pd.DataFrame(np.squeeze(cl_2_med_rna),index=Rna.columns[1:])
med_3_rna = pd.DataFrame(np.squeeze(cl_3_med_rna),index=Rna.columns[1:])

med_1_met = pd.DataFrame(np.squeeze(cl_1_med_met),index=Rna.columns[1:])
med_2_met = pd.DataFrame(np.squeeze(cl_2_med_met),index=Rna.columns[1:])
med_3_met = pd.DataFrame(np.squeeze(cl_3_med_met),index=Rna.columns[1:])

med_1_acc = pd.DataFrame(np.squeeze(cl_1_med_acc),index=Rna.columns[1:])
med_2_acc = pd.DataFrame(np.squeeze(cl_2_med_acc),index=Rna.columns[1:])
med_3_acc = pd.DataFrame(np.squeeze(cl_3_med_acc),index=Rna.columns[1:])

In [ ]:
#med_1_rna_rng.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/med_1_rna_rng_scMVI.csv",header=True,sep='\t')
#med_2_rna_rng.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/med_2_rna_rng_scMVI.csv",header=True,sep='\t')
#med_3_rna_rng.to_csv(Root_Folder + code_parent_folder +"/scMVI_DDLFC_meta/med_3_rna_rng_scMVI.csv",header=True,sep='\t')

#med_1_rna.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_1_scMVI_all_rna.csv",header=True,sep='\t')
#med_2_rna.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_2_scMVI_all_rna.csv",header=True,sep='\t')
#med_3_rna.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_3_scMVI_all_rna.csv",header=True,sep='\t')

#med_1_met.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_1_scMVI_all_met.csv",header=True,sep='\t')
#med_2_met.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_2_scMVI_all_met.csv",header=True,sep='\t')
#med_3_met.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_3_scMVI_all_met.csv",header=True,sep='\t')

#med_1_acc.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_1_scMVI_all_acc.csv",header=True,sep='\t')
#med_2_acc.to_csv(Root_Folder + code_parent_folder + "/scMVI_DDLFC_meta/Diff_Dir_2_scMVI_all_acc.csv",header=True,sep='\t')